In [ ]:
!pip install --upgrade tensorflow-io
import tensorflow as tf
import tensorflow_io as tfio
tf.__version__

In [2]:
import pandas as pd,numpy as np,os,plotly.graph_objects as go,tensorflow as tf
from tensorflow import keras
from scipy.ndimage import zoom
from plotly.subplots import make_subplots

In [4]:
import os,numpy as np
x_train=[];y_train=[];w_train=[];x_val=[];y_val=[];w_val=[]
for i in np.sort(os.listdir('/kaggle/input/zscores/train/X/')):
    temp=np.load('/kaggle/input/zscores/train/X/'+i)
    x_train.append(temp[:576])

for i in np.sort(os.listdir('/kaggle/input/zscores/train/Y/')):
    temp=np.load('/kaggle/input/zscores/train/Y/'+i)
    temp32=zoom(temp,[2,2,2]+[1],mode='reflect',order=1).astype('float32')
    y_train.append(temp32)

for i in np.sort(os.listdir('/kaggle/input/zscores/train/W/')):
    temp=np.load('/kaggle/input/zscores/train/W/'+i)
    temp32=zoom(temp,[2,2,2]+[1],mode='reflect',order=1).astype('float32')
    w_train.append(temp32)

for i in np.sort(os.listdir('/kaggle/input/zscores/val/X/')):
    temp=np.load('/kaggle/input/zscores/val/val/X/'+i)
    x_val.append(temp[:576])

for i in np.sort(os.listdir('/kaggle/input/zscores/val/val/Y/')):
    temp=np.load('/kaggle/input/zscores/val/val/Y/'+i)
    temp32=zoom(temp,[2,2,2]+[1],mode='reflect',order=1).astype('float32')
    y_val.append(temp32)

for i in np.sort(os.listdir('/kaggle/input/zscores/val/val/W/')):
    temp=np.load('/kaggle/input/zscores/val/val/W/'+i)
    temp32=zoom(temp,[2,2,2]+[1],mode='reflect',order=1).astype('float32')
    w_val.append(temp32)

In [5]:
w_train[0].shape

(32, 32, 16, 2)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Concatenate, MaxPooling3D, UpSampling3D, Multiply, Dropout, ZeroPadding2D, Flatten, Conv2D, Conv3D, Conv2DTranspose, Conv3DTranspose, BatchNormalization, Activation, Reshape, Concatenate, Dense
from tensorflow.keras.optimizers import Adamax, RMSprop
import numpy as np


def weighted_mse(y_true, y_pred, weights):
    sq_errors = tf.math.square(y_pred-y_true)
    weighted_sq_errors = sq_errors*weights
    return weighted_sq_errors


def fdnet(input_shape, output_shape, n_filters=64, kernel_size=5, fc_dropout=0.2, ac1='tanh', ac='relu', learning_rate=0.0001):

    input_meas = keras.Input(shape=input_shape)
    y_true = keras.Input(shape=output_shape)
    weights = keras.Input(shape=output_shape)

    x = Flatten()(input_meas)
    x = Dense(np.product(output_shape), activation=ac1)(x)
    x = Dropout(fc_dropout)(x)
    x = Reshape(output_shape)(x)

    x = Conv3D(filters=n_filters, kernel_size=kernel_size, activation=ac, padding='same')(x)
    x = MaxPooling3D((2,2,2))(x)
    x = Conv3D(filters=n_filters, kernel_size=kernel_size, activation=ac, padding='same')(x)
    x = Conv3DTranspose(n_filters, (2, 2, 2), strides=2, padding='same')(x)
    x = Conv3D(filters=n_filters, kernel_size=kernel_size, activation=ac, padding='same')(x)

    y_pred = Conv3D(filters=2, kernel_size=1, strides=1, padding='same', activation=ac)(x)

    model = keras.Model([input_meas, y_true, weights], y_pred)
    model.add_loss(weighted_mse(y_true, y_pred, weights))
    model.compile(optimizer=Adamax(learning_rate = learning_rate))


    final_model = keras.Model(input_meas, y_pred)

    return model, final_model


model_name='zscore_fdnet_undersampled_Amp32_Adamax'
model, final_model = fdnet(x_train[0].shape,y_train[0].shape)

history = model.fit([np.array(x_train),np.array(y_train),np.array(w_train)],None,
                    batch_size=64,verbose = 3,epochs=1000,
                    validation_data=[[np.array(x_val),np.array(y_val),np.array(w_val)],None],
                    callbacks = [EarlyStopping(monitor='val_loss',
                                mode='min',
                                restore_best_weights=True,
                                verbose=3,patience=60)])



model.save('/kaggle/working/'+model_name)
model.save('/kaggle/working/zscore_fdnet_undersampled_Amp32_Adamax.h5')
train_losses = history.history['loss']
val_losses = history.history['val_loss']
#vis_train_loss(history,title='losses',savename='/kaggle/working/models/training_losses')

In [ ]:
final_model.summary()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(train_losses,label='Training loss',color='skyblue')
plt.plot(val_losses,label='Validation loss',color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss vs Validation Loss')
plt.legend()
plt.savefig('loss_plot_Amp32')
plt.show()

In [ ]:
def save_losses_to_file(train_losses, val_losses, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Epoch', 'Train Loss', 'Validation Loss']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for epoch, (train_loss, val_loss) in enumerate(zip(train_losses, val_losses), start=1):
            writer.writerow({'Epoch': epoch, 'Train Loss': train_loss, 'Validation Loss': val_loss})
            
save_losses_to_file(train_losses, val_losses, 'losses_amp32.csv')

In [ ]:
!zip -r zscore_fdnet_undersampled_Amp32_Adamax.zip '/kaggle/working/zscore_fdnet_undersampled_Amp32_Adamax'
from IPython.display import FileLink
FileLink('zscore_fdnet_undersampled_Amp32_Adamax.zip')

In [ ]:
import pickle
with open('/kaggle/input/zscores/factors.p', 'rb') as f:
    factor = pickle.load(f)
factors={}
factors['mean']=factor[0]
factors['std']=factor[1]
factors['mua_min']=factor[2]
factors['mua_range']=factor[3]
factors['mus_min']=factor[4]
factors['mus_range']=factor[5]
factors['dwn_factor']=factor[6]
factors['roi_factor']=factor[7]
factors

In [ ]:
!pip install -U Kaleido

In [ ]:
import scipy.io, numpy as np,plotly.io as pio
from scipy.ndimage import zoom
from plotly.subplots import make_subplots
from skimage.metrics import structural_similarity as compare_ssim
import matplotlib.pyplot as plt

In [ ]:
file_name='D10_1.npy'
depth=file_name.split('_')[0][1:]
temp=np.load('/kaggle/input/test-disks-zscores-x/X/X/'+file_name)
y_pred=final_model.predict(temp[:576].reshape(1,-1))
y_pred_reshaped=np.squeeze(y_pred,0)
rescaled_y_pred_mua=(y_pred_reshaped[:,:,:,0]*factors['mua_range'])+factors['mua_min']
rescaled_y_pred_mus=(y_pred_reshaped[:,:,:,1]*factors['mus_range'])+factors['mus_min']

temp_dict=scipy.io.loadmat('/kaggle/input/test-disks/test_disks/'+file_name[:-4]+'.mat')
#Y=zoom(temp_dict['target'],factors['dwn_factor']+[1],mode='reflect',order=1).astype('float32')
temp_y_mua=temp_dict['target'][:,:,:,0]
temp_y_mus=temp_dict['target'][:,:,:,1]
data_range_mua=np.max([np.max(rescaled_y_pred_mua),np.max(temp_y_mua)])-np.min([np.min(rescaled_y_pred_mua),np.min(temp_y_mua)])
ssim_value_mua = compare_ssim(rescaled_y_pred_mua, temp_y_mua, channel_axis=-1,data_range=data_range_mua)
data_range_mus=np.max([np.max(rescaled_y_pred_mus),np.max(temp_y_mus)])-np.min([np.min(rescaled_y_pred_mus),np.min(temp_y_mus)])
ssim_value_mus = compare_ssim(rescaled_y_pred_mus, temp_y_mus, channel_axis=-1,data_range=data_range_mus)


'mua'
''
legend_coord=[[0.4,0.78],[0.98,0.78],[0.4,0.2],[0.98,0.2]]
X, Y, Z = np.mgrid[:16, :16, :8]

fig1 = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=rescaled_y_pred_mua.flatten(),
    isomin=np.min(np.min(rescaled_y_pred_mua))+(0*np.min(np.min(rescaled_y_pred_mua))),
    isomax=np.max(np.max(rescaled_y_pred_mua)),
    opacity=0.5,
    surface_count=6,
    colorbar=dict(len=0.4,x=legend_coord[0][0],y=legend_coord[0][1])
    ))
fig1.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
''
''
''
'Ground truth-mua'
''
X, Y, Z = np.mgrid[:16, :16, :8]

fig2 = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=temp_y_mua.flatten(),
    isomin=np.min(np.min(temp_y_mua))+(0*np.min(np.min(temp_y_mua))),
    isomax=np.max(np.max(temp_y_mua)),
    opacity=0.5,
    surface_count=6,
    colorbar=dict(len=0.4,x=legend_coord[1][0],y=legend_coord[1][1])
    ))
fig2.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
''
''
''
'mus'
''
X, Y, Z = np.mgrid[:16, :16, :8]

fig3 = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=rescaled_y_pred_mus.flatten(),
    isomin=np.min(np.min(rescaled_y_pred_mus))+(0*np.min(np.min(rescaled_y_pred_mus))),
    isomax=np.max(np.max(rescaled_y_pred_mus)),
    opacity=0.5,
    surface_count=6,
    colorbar=dict(len=0.4,x=legend_coord[2][0],y=legend_coord[2][1])
    ))
fig3.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
''
''
''
'Ground truth-mus'
''
X, Y, Z = np.mgrid[:16, :16, :8]

fig4 = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=temp_y_mus.flatten(),
    isomin=np.min(np.min(temp_y_mus))+(0*np.min(np.min(temp_y_mus))),
    isomax=np.max(np.max(temp_y_mus)),
    opacity=0.5,
    surface_count=6,
    colorbar=dict(len=0.4,x=legend_coord[3][0],y=legend_coord[3][1])
    ))
fig4.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
''
''
''
fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'scene'}, {'type': 'scene'}],[{'type': 'scene'},{'type': 'scene'}]])
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)
for trace in fig3.data:
    fig.add_trace(trace, row=2, col=1)
for trace in fig4.data:
    fig.add_trace(trace, row=2, col=2)
fig.update_layout(title_text='Depth - '+depth+' mm',width=1000, height=800, annotations=[
        dict(text="Amplitude Only-mua<br>SSIM: {:.6f}".format(ssim_value_mua), x=0.2, y=1, xref="paper", yref="paper", showarrow=False),
        dict(text="Ground truth-mua", x=0.8, y=1, xref="paper", yref="paper", showarrow=False),
        dict(text="Amplitude Only-mus<br>SSIM: {:.6f}".format(ssim_value_mus), x=0.2, y=0.45, xref="paper", yref="paper", showarrow=False),
        dict(text="Ground truth-mus", x=0.8, y=0.45, xref="paper", yref="paper", showarrow=False)
    ])
pio.write_image(fig, "/kaggle/working/Amp_32_"+depth+".png",format="png")
pio.write_image(fig, "/kaggle/working/Amp_32_"+depth+".svg",format="svg")
fig.show()

In [ ]:
#mua-LOCA & ER
import numpy as np
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt

def get_lesion_centroids(volume, threshold=0.010):
    binary_mask = volume > threshold
    labeled, num_features = ndi.label(binary_mask)
    centroids = []
    for i in range(1, num_features + 1):
        lesion_data = labeled == i
        coords = np.argwhere(lesion_data)
        centroid = coords.mean(axis=0)
        centroids.append(centroid)
    return centroids
def extract_lesion_roi(volume, threshold):
    binary_mask = volume > threshold
    labeled, num_features = ndi.label(binary_mask)
    rois = []
    for i in range(1, num_features + 1):
        lesion_data = volume * (labeled == i)
        if np.any(lesion_data):
            rois.append(lesion_data)
    return rois

def compute_localization_error(predicted_centroid, ground_truth_centroid):
    return np.linalg.norm(np.array(predicted_centroid) - np.array(ground_truth_centroid))

def compute_effective_resolution(predicted_roi, ground_truth_centroid):
    coords = np.argwhere(predicted_roi)
    distances = [np.linalg.norm(coord - ground_truth_centroid) for coord in coords]
    max_distance = max(distances)
    # Diameter of the sphere
    return 2 * max_distance

# Example usage:

predicted_volume = rescaled_y_pred_mua  # Your predicted volume
ground_truth_volume = temp_y_mua  # Your ground truth volume

threshold = 0.008  # adjust as per your requirement
predicted_centroids = get_lesion_centroids(predicted_volume, threshold)
ground_truth_centroids = get_lesion_centroids(ground_truth_volume, threshold)

predicted_rois = extract_lesion_roi(predicted_volume, threshold)
ground_truth_rois = extract_lesion_roi(ground_truth_volume, threshold)

localization_errors = []
effective_resolutions = []

for i, (predicted_roi, gt_roi) in enumerate(zip(predicted_rois, ground_truth_rois)):
    print(f"Processing lesion {i+1}...")

    # Compute localization error
    localization_error = compute_localization_error(predicted_centroids[i], ground_truth_centroids[i])
    print(f"Localization Error: {localization_error}")
    localization_errors.append(localization_error)

    # Compute effective resolution
    effective_res = compute_effective_resolution(predicted_roi, ground_truth_centroids[i])
    print(f"Effective Resolution: {effective_res}")
    effective_resolutions.append(effective_res)
    
lesions = list(range(1, len(localization_errors)+1))

In [ ]:
#mua-FWHM
import numpy as np
import scipy.ndimage as ndi
from scipy.signal import find_peaks
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt

def extract_lesion_roi(volume, threshold):
    binary_mask = volume > threshold
    labeled, num_features = ndi.label(binary_mask)
    rois = []
    for i in range(1, num_features + 1):
        lesion_data = volume * (labeled == i)
        if np.any(lesion_data):
            rois.append(lesion_data)
    return rois

def extract_profiles(roi):
    coords = np.argwhere(roi)
    centroid = coords.mean(axis=0).astype(int)
    profile_x = roi[:,centroid[1], centroid[2]]
    profile_y = roi[centroid[0], :, centroid[2]]
    profile_z= roi[centroid[0], centroid[1],:]
    return profile_x, profile_y, profile_z

def compute_fwhm(profile):
    half_max = np.max(profile) / 2.0
    spline = UnivariateSpline(range(len(profile)), profile - half_max, s=0)
    roots = spline.roots()
    if len(roots) >= 2:
        return abs(roots[-1] - roots[0])
    return 0

# Example usage:
volume1 = rescaled_y_pred_mua  # Replace with your volume
volume2 = temp_y_mua  # Replace with your volume

threshold = 0.008  # Adjust based on your data

volumes = [volume1, volume2]
volume_name=['Predicted','Ground Truth']
roi_names=['lesion 1', 'lesion 2']
fwhms = []

for volume,vname in zip(volumes,volume_name):
    rois = extract_lesion_roi(volume, threshold)
    fwhm_values = []

    for roi,rname in zip(rois,roi_names):
        profiles = extract_profiles(roi)
        profile_names = ["x-profile", "y-profile", "z-profile"]
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        for idx, (profile, name) in enumerate(zip(profiles, profile_names)):
            if profile.ndim > 1:
                interpolated_profile = profile.sum(axis=1)
            else:
                interpolated_profile = profile
            fwhm = compute_fwhm(interpolated_profile)
            fwhm_values.append(fwhm)

            x = np.linspace(0, len(interpolated_profile)-1, 1000)
            spline = UnivariateSpline(range(len(interpolated_profile)), interpolated_profile - np.max(interpolated_profile) / 2.0, s=0)
            axes[idx].plot(interpolated_profile, label='Profile')
            axes[idx].plot(x, spline(x) + np.max(interpolated_profile) / 2.0, 'r--', label='Interpolated')
            axes[idx].axhline(y=np.max(interpolated_profile)/2.0, color='g', linestyle='--', label='Half Maximum')
            axes[idx].text(0.6, 0.6, f'FWHM: {fwhm:.2f}', transform=axes[idx].transAxes, fontsize=9, bbox=dict(facecolor='white', alpha=0.6))  # Display FWHM on the graph
            axes[idx].set_title(name)
            axes[idx].set_xlabel("Position")
            axes[idx].set_ylabel("Intensity")
            legend = axes[idx].legend()
            
        
        fig.suptitle("Intensity Profile and Interpolated FWHM - "+"Mua - " + vname+" - "+rname)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig("amp_mua_"+f"{vname}_{rname}.png")
        plt.show()
        plt.close(fig)

    fwhms.append(fwhm_values)

print(fwhms)
fig.savefig("Amp32_fwhm_mua.png")

In [ ]:
#mus-LOCA & ER
import numpy as np
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt

def get_lesion_centroids(volume, threshold=0.010):
    binary_mask = volume > threshold
    labeled, num_features = ndi.label(binary_mask)
    centroids = []
    for i in range(1, num_features + 1):
        lesion_data = labeled == i
        coords = np.argwhere(lesion_data)
        centroid = coords.mean(axis=0)
        centroids.append(centroid)
    return centroids

def compute_localization_error(predicted_centroid, ground_truth_centroid):
    return np.linalg.norm(np.array(predicted_centroid) - np.array(ground_truth_centroid))

def compute_effective_resolution(predicted_roi, ground_truth_centroid):
    coords = np.argwhere(predicted_roi)
    distances = [np.linalg.norm(coord - ground_truth_centroid) for coord in coords]
    max_distance = max(distances)
    # Diameter of the sphere
    return 2 * max_distance

# Example usage:

predicted_volume = rescaled_y_pred_mus  # Your predicted volume
ground_truth_volume = temp_y_mus  # Your ground truth volume

threshold = 0.8 # adjust as per your requirement
predicted_centroids = get_lesion_centroids(predicted_volume, threshold)
ground_truth_centroids = get_lesion_centroids(ground_truth_volume, threshold)

predicted_rois = extract_lesion_roi(predicted_volume, threshold)
ground_truth_rois = extract_lesion_roi(ground_truth_volume, threshold)

localization_errors = []
effective_resolutions = []

for i, (predicted_roi, gt_roi) in enumerate(zip(predicted_rois, ground_truth_rois)):
    print(f"Processing lesion {i+1}...")

    # Compute localization error
    localization_error = compute_localization_error(predicted_centroids[i], ground_truth_centroids[i])
    print(f"Localization Error: {localization_error}")
    localization_errors.append(localization_error)

    # Compute effective resolution
    effective_res = compute_effective_resolution(predicted_roi, ground_truth_centroids[i])
    print(f"Effective Resolution: {effective_res}")
    effective_resolutions.append(effective_res)
    
lesions = list(range(1, len(localization_errors)+1))


In [ ]:
#mus-FWHM
import numpy as np
import scipy.ndimage as ndi
from scipy.signal import find_peaks
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt

def extract_lesion_roi(volume, threshold):
    binary_mask = volume > threshold
    labeled, num_features = ndi.label(binary_mask)
    rois = []
    for i in range(1, num_features + 1):
        lesion_data = volume * (labeled == i)
        if np.any(lesion_data):
            rois.append(lesion_data)
    return rois

def extract_profiles(roi):
    coords = np.argwhere(roi)
    centroid = coords.mean(axis=0).astype(int)
    profile_x = roi[:,centroid[1], centroid[2]]
    profile_y = roi[centroid[0], :, centroid[2]]
    profile_z= roi[centroid[0], centroid[1],:]
    return profile_x, profile_y, profile_z

def compute_fwhm(profile):
    half_max = np.max(profile) / 2.0
    spline = UnivariateSpline(range(len(profile)), profile - half_max, s=0)
    roots = spline.roots()
    if len(roots) >= 2:
        return abs(roots[-1] - roots[0])
    return 0

# Example usage:
volume1 = rescaled_y_pred_mus  # Replace with your volume
volume2 = temp_y_mus  # Replace with your volume

threshold = 0.8  # Adjust based on your data

volumes = [volume1, volume2]
volume_name=['Predicted','Ground Truth']
roi_names=['lesion 1', 'lesion 2']
fwhms = []

for volume,vname in zip(volumes,volume_name):
    rois = extract_lesion_roi(volume, threshold)
    fwhm_values = []

    for roi,rname in zip(rois,roi_names):
        profiles = extract_profiles(roi)
        profile_names = ["x-profile", "y-profile", "z-profile"]
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        for idx, (profile, name) in enumerate(zip(profiles, profile_names)):
            if profile.ndim > 1:
                interpolated_profile = profile.sum(axis=1)
            else:
                interpolated_profile = profile
            fwhm = compute_fwhm(interpolated_profile)
            fwhm_values.append(fwhm)

            x = np.linspace(0, len(interpolated_profile)-1, 1000)
            spline = UnivariateSpline(range(len(interpolated_profile)), interpolated_profile - np.max(interpolated_profile) / 2.0, s=0)
            axes[idx].plot(interpolated_profile, label='Profile')
            axes[idx].plot(x, spline(x) + np.max(interpolated_profile) / 2.0, 'r--', label='Interpolated')
            axes[idx].axhline(y=np.max(interpolated_profile)/2.0, color='g', linestyle='--', label='Half Maximum')
            axes[idx].text(0.6, 0.6, f'FWHM: {fwhm:.2f}', transform=axes[idx].transAxes, fontsize=9, bbox=dict(facecolor='white', alpha=0.6))  # Display FWHM on the graph
            axes[idx].set_title(name)
            axes[idx].set_xlabel("Position")
            axes[idx].set_ylabel("Intensity")
            legend = axes[idx].legend()
            
        
        fig.suptitle("Intensity Profile and Interpolated FWHM - "+" Mus - " + vname+" - "+rname)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig("amp_mus_"+f"{vname}_{rname}.png")
        plt.show()
        plt.close(fig)

    fwhms.append(fwhm_values)

print(fwhms)